In [1]:
import sys
sys.path.append('../../bstadt/NeuroDataResource')
sys.path.append('../../bstadt/Util')

import pandas as pd
import timeit
from functools import partial
from NeuroDataResource import NeuroDataResource

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [2]:
with open('./data/api_key.txt', 'r') as f:
    api_key = f.readline()

In [3]:
channel_list = [
    {'name': 'annotations', 'dtype': 'uint64'},
    {'name': 'DAPI1st', 'dtype': 'uint8'},
    {'name': 'DAPI2nd', 'dtype': 'uint8'},
    {'name': 'DAPI3rd', 'dtype': 'uint8'},
    {'name': 'EM25K', 'dtype': 'uint8'},
    {'name': 'EM2k', 'dtype': 'uint8'},
    {'name': 'GABA488', 'dtype': 'uint8'},
    {'name': 'GAD647', 'dtype': 'uint8'},
    {'name': 'gephyrin594', 'dtype': 'uint8'},
    {'name': 'GS594', 'dtype': 'uint8'},
    {'name': 'MBP488', 'dtype': 'uint8'},
    {'name': 'NR1594', 'dtype': 'uint8'},
    {'name': 'PSD95_488', 'dtype': 'uint8'},
    {'name': 'Synapsin647', 'dtype': 'uint8'},
    {'name': 'VGluT1_647', 'dtype': 'uint8'}
]

In [4]:
resource = NeuroDataResource('api.boss.neurodata.io',
                               api_key,
                               'collman',
                               'collman15v2',
                               channel_list)

In [5]:
def download_chunk_time(resource, channels, z_range, xy_range,  n):
    """
    Parameters
    ----------
    resource : NeuroDataResource object
    channels : list of strings
    z_range : list of ints
    xy_range : list of ints
    n : int
        Number of times to repeat each download
    """
    
    out = pd.DataFrame(columns=['z', 'y', 'x', 'time', 'channel'])
    
    for channel in channels:
        for z in z_range:
            for xy in xy_range:
                for i in range(0, 3):
                    x_dim = y_dim = [xy * i, xy * i + xy]
                    z_dim = [z * i, z * i + z]

                    #max x is 6306
                    if x_dim[-1] > 6306:
                        x_dim = [0, xy]

                    #since max y is 4518
                    if y_dim[-1] > 4518:
                        y_dim = [0, xy]

                    #since max z is 27
                    if z_dim[-1] > 27:
                        z_dim = [0, z]                     

                    t = timeit.Timer(partial(resource.get_cutout, channel, z_dim, y_dim, x_dim))
                    avg_time = t.timeit(n)/n
                    print('Testing: ', '\txy: ', xy, '\tz: ', z, '\tavg_time: ', avg_time, '\tchannel: ', channel)
                    out.loc[len(out)] = [z, xy, xy, avg_time, channel]
    
    return out

In [6]:
time_df = download_chunk_time(resource, 
                              ['DAPI1st', 'DAPI2nd', 'DAPI3rd'], 
                              [2**x for x in range(0, 5)], #1-16
                              [2**x for x in range(5, 13)], #32-4096
                              3)

Testing:  	xy:  32 	z:  1 	avg_time:  0.21156952132393295 	channel:  DAPI1st
Testing:  	xy:  32 	z:  1 	avg_time:  0.14582100566864634 	channel:  DAPI1st
Testing:  	xy:  32 	z:  1 	avg_time:  0.15060034133299874 	channel:  DAPI1st
Testing:  	xy:  64 	z:  1 	avg_time:  0.14241496265943474 	channel:  DAPI1st
Testing:  	xy:  64 	z:  1 	avg_time:  0.15237485266212994 	channel:  DAPI1st
Testing:  	xy:  64 	z:  1 	avg_time:  0.1511808959961248 	channel:  DAPI1st
Testing:  	xy:  128 	z:  1 	avg_time:  0.3430312656661651 	channel:  DAPI1st
Testing:  	xy:  128 	z:  1 	avg_time:  0.1647761426672029 	channel:  DAPI1st
Testing:  	xy:  128 	z:  1 	avg_time:  0.14192749932408333 	channel:  DAPI1st
Testing:  	xy:  256 	z:  1 	avg_time:  0.14529245199325183 	channel:  DAPI1st
Testing:  	xy:  256 	z:  1 	avg_time:  0.15336180633554855 	channel:  DAPI1st
Testing:  	xy:  256 	z:  1 	avg_time:  0.19073044633842073 	channel:  DAPI1st
Testing:  	xy:  512 	z:  1 	avg_time:  0.36278690333710983 	channel:  DAP

Testing:  	xy:  256 	z:  16 	avg_time:  0.19408751166580865 	channel:  DAPI1st
Testing:  	xy:  256 	z:  16 	avg_time:  0.27087740233400837 	channel:  DAPI1st
Testing:  	xy:  512 	z:  16 	avg_time:  0.2425763663292552 	channel:  DAPI1st
Testing:  	xy:  512 	z:  16 	avg_time:  0.4051244196695431 	channel:  DAPI1st
Testing:  	xy:  512 	z:  16 	avg_time:  0.403341818668802 	channel:  DAPI1st
Testing:  	xy:  1024 	z:  16 	avg_time:  1.2586249493372936 	channel:  DAPI1st
Testing:  	xy:  1024 	z:  16 	avg_time:  1.2612128596714076 	channel:  DAPI1st
Testing:  	xy:  1024 	z:  16 	avg_time:  1.2910650810032773 	channel:  DAPI1st
Testing:  	xy:  2048 	z:  16 	avg_time:  3.914096313334691 	channel:  DAPI1st
Testing:  	xy:  2048 	z:  16 	avg_time:  4.478305605327478 	channel:  DAPI1st
Testing:  	xy:  2048 	z:  16 	avg_time:  3.0577898449943555 	channel:  DAPI1st
Testing:  	xy:  4096 	z:  16 	avg_time:  20.566825628668692 	channel:  DAPI1st
Testing:  	xy:  4096 	z:  16 	avg_time:  20.72552863299885

Testing:  	xy:  2048 	z:  8 	avg_time:  1.3538721623384238 	channel:  DAPI2nd
Testing:  	xy:  4096 	z:  8 	avg_time:  6.167400600999827 	channel:  DAPI2nd
Testing:  	xy:  4096 	z:  8 	avg_time:  6.280380107995977 	channel:  DAPI2nd
Testing:  	xy:  4096 	z:  8 	avg_time:  8.509141218334358 	channel:  DAPI2nd
Testing:  	xy:  32 	z:  16 	avg_time:  0.12758844100365727 	channel:  DAPI2nd
Testing:  	xy:  32 	z:  16 	avg_time:  0.12744311934026578 	channel:  DAPI2nd
Testing:  	xy:  32 	z:  16 	avg_time:  0.12850191033794545 	channel:  DAPI2nd
Testing:  	xy:  64 	z:  16 	avg_time:  0.12840025300586907 	channel:  DAPI2nd
Testing:  	xy:  64 	z:  16 	avg_time:  0.12327654332815048 	channel:  DAPI2nd
Testing:  	xy:  64 	z:  16 	avg_time:  0.13569632199748108 	channel:  DAPI2nd
Testing:  	xy:  128 	z:  16 	avg_time:  0.12623390467100157 	channel:  DAPI2nd
Testing:  	xy:  128 	z:  16 	avg_time:  0.12965503866629055 	channel:  DAPI2nd
Testing:  	xy:  128 	z:  16 	avg_time:  0.13095044799653502 	chan

Testing:  	xy:  128 	z:  8 	avg_time:  0.12805757932558967 	channel:  DAPI3rd
Testing:  	xy:  128 	z:  8 	avg_time:  0.13011383367120288 	channel:  DAPI3rd
Testing:  	xy:  128 	z:  8 	avg_time:  0.11961730699598168 	channel:  DAPI3rd
Testing:  	xy:  256 	z:  8 	avg_time:  0.13569084466629042 	channel:  DAPI3rd
Testing:  	xy:  256 	z:  8 	avg_time:  0.14117461300338618 	channel:  DAPI3rd
Testing:  	xy:  256 	z:  8 	avg_time:  0.21571558533469215 	channel:  DAPI3rd
Testing:  	xy:  512 	z:  8 	avg_time:  0.14463143066192666 	channel:  DAPI3rd
Testing:  	xy:  512 	z:  8 	avg_time:  0.19063188600315092 	channel:  DAPI3rd
Testing:  	xy:  512 	z:  8 	avg_time:  0.2543202849919908 	channel:  DAPI3rd
Testing:  	xy:  1024 	z:  8 	avg_time:  0.2994459366697508 	channel:  DAPI3rd
Testing:  	xy:  1024 	z:  8 	avg_time:  0.36117754433265264 	channel:  DAPI3rd
Testing:  	xy:  1024 	z:  8 	avg_time:  0.5456966559965318 	channel:  DAPI3rd
Testing:  	xy:  2048 	z:  8 	avg_time:  1.033483030332718 	chann

In [7]:
# Test downloading the entire set using different z
y_range = [0, 4518]
x_range = [0, 6306]
n = 2

for z in [1, 2, 4, 8, 16, 27]:
    t = timeit.Timer(partial(resource.get_cutout, 'DAPI1st', [0, z], y_range, x_range))
    avg_time = t.timeit(n)/n
    time_df.loc[len(time_df)] = [z, 4518, 6306, avg_time, 'DAPI1st']

In [8]:
time_df.tail()

,z,y,x,time,channel
361,2,4518,6306,7.745348,DAPI1st
362,4,4518,6306,17.103899,DAPI1st
363,8,4518,6306,21.007157,DAPI1st
364,16,4518,6306,31.309585,DAPI1st
365,27,4518,6306,54.177639,DAPI1st


In [9]:
time_df_dapi1st = time_df.loc[(time_df['channel'] == 'DAPI1st') & (time_df['x'] != 6306)]
DAPI1st = go.Scatter3d(
    x = time_df_dapi1st['x'],
    y = time_df_dapi1st['z'],
    z = time_df_dapi1st['time'],
    mode='markers',
    marker = dict(size=12, color=100, opacity=.5),
    name='DAPI1st')

time_df_dapi2nd = time_df.loc[time_df['channel'] == 'DAPI2nd']
DAPI2nd = go.Scatter3d(
    x = time_df_dapi2nd['x'],
    y = time_df_dapi2nd['z'],
    z = time_df_dapi2nd['time'],
    mode='markers',
    marker = dict(size=12, color=200, opacity=0.5),
    name='DAPI2nd')

time_df_dapi3rd = time_df.loc[time_df['channel'] == 'DAPI3rd']
DAPI3rd = go.Scatter3d(
    x = time_df_dapi3rd['x'],
    y = time_df_dapi3rd['z'],
    z = time_df_dapi3rd['time'],
    mode='markers',
    marker=dict(size=12, color=300, opacity=0.5),
    name='DAPI3rd')

data = [DAPI1st, DAPI2nd, DAPI3rd]
layout = go.Layout(margin=dict(l=0, r=0, t=0, b=0),
                    scene = dict(
                        xaxis = dict(title='XY'),
                        yaxis = dict(title='z'),
                        zaxis = dict(title='Time'),)
                  )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [10]:
time_df['volume'] = time_df['x'] * time_df['y'] * time_df['z']

volume_time = time_df.groupby(['volume'])['time'].mean().to_frame()

channel_volume = 6306*4518*27
for index, row in volume_time.iterrows():
    volume_time.loc[index, 'number_of_chunks'] = channel_volume/index
    volume_time.loc[index, 'total_time'] = channel_volume/index * row['time']

In [11]:
volume_time

,time,number_of_chunks,total_time
volume,,,
1024,0.165351,751214.566406,124213.789413
2048,0.137893,375607.283203,51793.435465
4096,0.137323,187803.641602,25789.773470
8192,0.168100,93901.820801,15784.860343
16384,0.166495,46950.910400,7817.092081
32768,0.132664,23475.455200,3114.339496
65536,0.163033,11737.727600,1913.635071
131072,0.139656,5868.863800,819.623074
262144,0.173601,2934.431900,509.419205


In [12]:
time_df.to_csv('./data/time_df.csv', index=False)
volume_time.to_csv('./data/volume_time.csv', index=False)